### Installations, if needed

In [ ]:
# Install the right version of spaCy
!pip install spacy==2.0.12 # Above 2.0.12 doesn't seem work with the neuralcoref resolution (at least 2.0.13 and 2.0.16 don't)

In [ ]:
# Install the large Neural Coref model
!pip install https://github.com/huggingface/neuralcoref-models/releases/download/en_coref_lg-3.0.0/en_coref_lg-3.0.0.tar.gz # This is the coref language model

In [ ]:
# Install dill which we're using for pickling our Named Entity Type (NET) graphs
!pip install dill

### Importing Libraries

In [2]:
import spacy
import en_coref_lg
from spacy import displacy
import pandas as pd
import numpy as np
import networkx as nx
import time
import os
import dill as pickle
from collections import defaultdict

### Skip: Loading and previewing our export from OntoNotes5

In [ ]:
JSON_FILENAME = 'ner_output_1.json'
FILEPATH_TO_JSON = "onto_sql_output/"

onto_import = pd.read_json(FILEPATH_TO_JSON + JSON_FILENAME)

In [ ]:
onto_import.head()

### Load the Large Neural Coref spaCy Model

In [10]:
%time
nlp = en_coref_lg.load()

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 6.2 µs


#### Skip: Using it to loop through dependency parsing for a selection of sentences

In [15]:
# Establish which sentence rows we want to work through right now
SENT_MIN = 0
SENT_MAX = 500

# Check the time and start parsing via spaCy
start_time = time.time()
onto_import["spacy_parse"] = onto_import.loc[SENT_MIN:SENT_MAX,:].apply(lambda x: nlp(x["sentence_string"]), axis=1)

# Calculate the duration 
duration = time.time() - start_time
print("Applying the spaCy pipeline took {0:.2f} seconds".format(duration))

NameError: name 'onto_import' is not defined

### Skip: Viewing some example text, with highlighted named entities

In [ ]:
# Choose an entry integer to see its text and the parse below.
ENTRY = 490

displacy.render(onto_import.loc[ENTRY,"spacy_parse"], jupyter=True, style='ent')

#### Skip: Viewing dependencies

In [ ]:
displacy.render(onto_import.loc[ENTRY,"spacy_parse"], jupyter=True, style='dep')

#### Skip: Identifying some useful attributes of the spaCy tokens

In [ ]:
# This gives us the dependency type

onto_import.loc[490,"spacy_parse"][13].head

In [ ]:
# This gives us the token of its head, from which we can call other attributes of the head.

onto_import.loc[490,"spacy_parse"][13].head.i

In [ ]:
# If we just want the string of the head, that's here:

onto_import.loc[0,"spacy_parse"][32].head.text

In [ ]:
# If we want the index of the head within the sentence (to find when a multi-word NE
# depends on something outside of that NE phrase)

onto_import.loc[0,"spacy_parse"][32].head.i

### Skip: Creating a Dictionary with a Graph for Each NER Type

#### Skip: Establish dictionary to hold graphs and define a function to graph a row 

In [3]:
graphs_dict = dict()

def graph_row(df_row):
    
    NO_OF_GENERATIONS = 2
    
    root_node = str(df_row["ner_type"])

    # Retrieve our Directed Graph for this NE Type or create a new one
    G = graphs_dict.get(root_node, nx.DiGraph())
    
    # For each row, add a node for the Named Entity's type
    G.add_node(root_node)
    node_weight = G.nodes[root_node].get('weight', 0)
    G.node[root_node]['weight'] = node_weight + 1
    G.node[root_node]['xlabel'] = G.node[root_node]['weight']

    # Let's assume the head is the last word of the phrase
    # and get that Token from the spaCy parse:
    
    head_index = df_row["ner_end_word_index"]
    head_of_phrase = df_row["spacy_parse"][head_index]
    
    nodes_needing_head_branches = [(head_of_phrase, root_node)]
    next_head_nodes = []
    nodes_needing_child_branches = [(head_of_phrase, root_node)]
    next_child_nodes = []
    
    current_gen = 1
    
    while current_gen <= NO_OF_GENERATIONS:
        
        for (node, child_label) in nodes_needing_head_branches:
            try: 
                # Get the explanation of its dependency type in this usage
                relation = spacy.explain(node.dep_)

                # If no explanation, revert to the raw dependency type.
                if relation is None:
                    relation = node.dep_

                # Trying to catch and diagnose some problem cases
                elif relation == "punctuation":
                    print("NE '{1}' marked as punctuation in sentence '{0}'".format(df_row["sentence_string"], df_row["ner_string"]))
                    print(" --- ")
                elif relation == "determiner":
                    print("NE '{1}' marked as determiner in sentence '{0}'".format(df_row["sentence_string"], df_row["ner_string"]))
                    print(" --- ")

                # Object of preposition doesn't do much, so let's see what's on the other side of that.
                elif relation == "object of preposition":
                    relation = "head of prep phrase"
                    # move to the preposition so we get its head later on when adding node
                    node = node.head
                    
                print("relation: " + relation)
                
                if relation == 'ROOT':
                    continue
                
                # differentiating head-focused edges from child-focused edges
                intermediary_node_label = "head g{0} {1}".format(current_gen, relation)
                
                print("intermediary node label:" + intermediary_node_label)

                # Add a node for the relation, and connect that to the main entity.
                # Add the weights to the nodes and edges.
                G.add_node(intermediary_node_label)
                relation_weight = G.node[intermediary_node_label].get('weight', 0)
                G.node[intermediary_node_label]['weight'] = relation_weight + 1
                
                # Add a node from the dependency type to the entity's head and add weights
                norm = node.head.norm_
                G.add_node(norm)
                norm_edge_weight = G.node[norm].get('weight', 0)
                G.node[norm]['weight'] = norm_edge_weight + 1
                
                G.add_edge(child_label, intermediary_node_label, label="head")
                
                G.add_edge(intermediary_node_label, norm)
                
                # Add the next round for the next generation
                if node.head != node:
                    next_head_nodes.append((node.head, norm))
                
            except:
                print("passed in head")
                pass
            
            # Move the next round into the queue and clear it
            nodes_needing_head_branches = next_head_nodes
            next_head_nodes = []

            for (node, parent_label) in nodes_needing_child_branches:
                for child in node.children:

                    # Get the relation of the child node to this one.
                    relation = spacy.explain(child.dep_)

                    # If no explanation, revert to the raw dependency type.
                    if relation is None:
                        relation = child.dep_

                    if relation == 'punctuation':
                        continue

                    # Differentiate these relations from head relations
                    # and add the node and its weights
                    intermediary_node_label = "child g{0} {1}".format(current_gen, relation)
                    G.add_node(intermediary_node_label)
                    relation_weight = G.nodes[intermediary_node_label].get('weight', 0)
                    G.node[intermediary_node_label]['weight'] = relation_weight + 1
                    
                    # Add the child as normed, and add its edge and weights
                    child_norm = child.norm_
                    G.add_node(child_norm)
                    leaf_weight = G.node[child_norm].get('weight', 0)
                    G.node[child_norm]['weight'] = leaf_weight + 1

                    # add edge between the parent node and this relation and weights
                    G.add_edge(parent_label, intermediary_node_label, label="child")
                    
                    G.add_edge(intermediary_node_label, child_norm)

                    # Queue up the children for the next generation
                    for childs_child in child.children:
                        next_child_nodes.append((childs_child, child_norm))      
                
                # Move the children into the queue and clear it.
                nodes_needing_child_branches = next_child_nodes
                next_child_nodes = []
        
        # Increment the generation
        current_gen += 1

    graphs_dict[root_node] = G

#### Skip: Applying that to our selected rows of our dataframe

In [ ]:
%time
_ = onto_import.loc[SENT_MIN:SENT_MAX,:].apply(lambda x: graph_row(x), axis=1)

#### Skip: Calculate the Log Probability of the Named Entity Types

In [ ]:
def normalize_as_logp(successor_nodes, G):
    log_total = np.log(sum([G.node[node]['weight'] for node in successor_nodes]))
    for n in successor_nodes:
        G.node[n]['lp'] = np.log(G.node[n]['weight']) - log_total

def calc_lps(G):
    for node in nx.nodes(G):
        if G.successors(node):
            [normalize_as_logp(succ[1], G) for succ in nx.bfs_successors(G, node)]
    return G

In [ ]:
%time

log_graphs_dict = {key:calc_lps(value) for (key,value) in graphs_dict.items()}

#### Skip: Draw the graphs for the Named Entity Types

In [ ]:
%time

for key, value in graphs_dict.items():
    
    graph_filepath = 'NER_Type_Graphs/'
    graph_filename = 'G_{0}'.format(key)

    # Write our graph to DOT format to be read and visualized by GraphViz
    nx.drawing.nx_pydot.write_dot(value, "{0}.dot".format(graph_filepath + graph_filename))
    
    # Write the graphs via neato
    os.system("neato -Tsvg -Goverlap=false -Gsplines=true -Gsep=0.1 {0}.dot -o {0}.svg".format(graph_filepath + graph_filename))

### Graphing a Candidate Named Entity

In [3]:
# Defining helper functions to build the candidate graphs

def reconcile_ents_and_clusters(document_id, doc):
    """"Reconcile the coreference and entities lists into a
        a single dict of graphs to make.
        
        Keys are ent_id strings [document_id]:[start_word_index]:[end_word_index].
        Values are (spaCy.Span, graph_id) tuples."""
    
    # Keys are (start.idx, end.idx) tuples.
    # Values are (spaCy.Span, graph_id) tuples."
    occurence_ind  = {}
    
    if doc._.has_coref:
        cluster_offset = len(doc._.coref_clusters)
        for cluster_idx, cluster in enumerate(doc._.coref_clusters):
            for mention in cluster:
                key = ":".join([document_id,str(mention.start), str(mention.end -1)])
                occurence_ind[key] = (mention, cluster_idx)
                
    # Now let's see if each ent is in there. If not, we'll add it to
    # our cluster list.
    new_cluster_idx = 0
    
    for ent_ind, ent in enumerate(doc.ents):
        key = ":".join([document_id,str(ent.start), str(ent.end -1)])
        try:
            occurence_ind[key]
        except:
            occurence_ind[key] = (ent, cluster_offset + new_cluster_idx)
            new_cluster_idx += 1
    
    print("Length of occurence index for doc {0} is: {1}".format(document_id, len(occurence_ind)))
    
    return occurence_ind

def graph_entity(ent, doc, G, root_node):
    
    NO_OF_GENERATIONS = 2
    
    # Assume the head of the phrase, if it is a phrase, is the last word
    # in the phrase.
    head_of_phrase = ent[-1]
    
    nodes_needing_head_branches = [(head_of_phrase, root_node)]
    next_head_nodes = []
    nodes_needing_child_branches = [(head_of_phrase, root_node)]
    next_child_nodes = []
    
    current_gen = 1
    while current_gen <= NO_OF_GENERATIONS:
        
        for (node, child_label) in nodes_needing_head_branches:
            try: 
                # Get the explanation of its relation arc in this usage
                relation = spacy.explain(node.dep_)
                # If no explanation, revert to the raw dependency type.
                if relation is None:
                    relation = node.dep_
                    
                if relation == 'ROOT':
                    continue

                # Object of preposition doesn't do much, so let's see what's on the other side of that.
                elif relation == "object of preposition":
                    relation = "head of prep phrase"
                    # move to the preposition so we get its head later on when adding node
                    node = node.head
                    
                intermediary_node_label = "head g{0} {1}".format(current_gen, relation)

                # Add a node for the relation, and connect that to the main entity
                G.add_node(intermediary_node_label)
                G.add_edge(child_label, intermediary_node_label, label="head")

                # Add a node from the relation to the entity's head, and connect that
                # to the relation type
                normed_head = node.head.norm_
                #print("adding head node: {0}".format(normed_head))
                G.add_node(normed_head)
                G.add_edge(intermediary_node_label, normed_head)
                
                if node.head != node:
                    next_head_nodes.append((node.head, node.head.text))
            except Exception as ex:
                print("passed in head.\n{0}".format(ex))
                pass
            
        nodes_needing_head_branches = next_head_nodes
        next_head_nodes = []
        
        for (node, parent_label) in nodes_needing_child_branches:
            try:
                for child in node.children:
                    relation = spacy.explain(child.dep_)
                    # If no explanation, revert to the raw dependency type.
                    if relation is None:
                        relation = node.dep_
                        
                    elif relation == 'punctuation':
                        continue
                        
                    intermediary_node_label = "child g{0} {1}".format(current_gen, relation)

                    G.add_node(intermediary_node_label)
                    G.add_edge(parent_label, intermediary_node_label, label="child")
                    
                    normed_child = child.norm_
                    G.add_node(normed_child)
                    #print("adding child node: {0}".format(normed_child))
                    G.add_edge(intermediary_node_label, normed_child)
                    
                    for childs_child in child.children:
                        next_child_nodes.append((childs_child, childs_child.text))
            except Exception as ex:
                print("passed in child.\n{0}".format(ex))
                pass                
        nodes_needing_child_branches = next_child_nodes
        next_child_nodes = []
        
        # Increment the generation
        current_gen += 1
        
    return G
    
def graph_candidates_in_doc(document_id, candidate_text):
    
    doc = nlp(candidate_text)
    
    # Key of clustered_ents is ent_it. Value is (Span, graph_cluter_id)
    clustered_ents = reconcile_ents_and_clusters(document_id, doc)
    
    # Initialize a graph for each clustered_ent
    candidate_graphs = dict()
    
    for ent_id, (ent, graph_idx) in clustered_ents.items():
        
        # Make sure we have our root. No harm done if it already exists.
        # If it's a cluster, we get the Span of the most representative
        # mention in the cluster
        try:
            root_node = doc._.coref_clusters[graph_idx].main.text
        # If it's not, we just use the ent name
        except:
            root_node = ent.text
        
        # Get the cluster's root node and existing graph from previous mentions
        # or create a new tuple of those.
        root_node, G, _ = candidate_graphs.get(graph_idx, (root_node, nx.DiGraph(), graph_idx))
        
        G.add_node(root_node)
        
        # A helper function adds the rest of the graph
        print("\nGraphing entity: {0}".format(ent.text))
        candidate_graphs[graph_idx] = (root_node, graph_entity(ent, doc, G, root_node), graph_idx)
    
    spacy_ent_graphs = {}
    
    # Filter back to just the spaCy ents
    for ent in doc.ents:
        key = ":".join([document_id,str(ent.start), str(ent.end - 1)])
        _, ent_graph_idx = clustered_ents[key]
        spacy_ent_graphs[key] = candidate_graphs[ent_graph_idx]
    
    #graph_entity(ent, doc) for ent in doc.ents]
    return spacy_ent_graphs

#### Skip: Applying the functions on a selected article

In [ ]:
TEST_ENTRY = 90
test_candidate_text = onto_import.loc[TEST_ENTRY,"document_text"]
print(test_candidate_text)

candidate_roots_and_graphs = graph_candidates_in_doc("test_doc_id",test_candidate_text)

In [ ]:
for ent_id in candidate_roots_and_graphs.values():
    print(ent_id)

#### Skip: Draw the candidate graphs for that article

In [ ]:
for key, (root, cand_graph, clust) in candidate_roots_and_graphs.items():
    
    graph_filepath = 'NER_Type_Graphs/'
    graph_filename = 'G_' + str(key)

    # Write our graph to DOT format to be read and visualized by GraphViz
    nx.drawing.nx_pydot.write_dot(cand_graph, "{0}.dot".format(graph_filepath + graph_filename))
    
    os.system("neato -Tsvg -Goverlap=false -Gsplines=true -Gsep=0.1 {0}.dot -o {0}.svg".format(graph_filepath + graph_filename))

## Similarity Between Graphs

In [219]:
def logsumexp(a):
    """Simple re-implementation of scipy.misc.logsumexp."""
    a_max = np.max(a)
    if a_max == -np.inf:
        return -np.inf
    sumexp = np.sum(np.exp(a - a_max))
    return np.log(sumexp) + a_max

def score_relation_and_children(nlp, cand_G, NET_G, cand_parent, net_options, cand_succ_dict, NET_succ_dict, sim_dict, parent_weight):
        
    # If this node has no further children,
    # compare it to its NET options
    if cand_parent not in cand_succ_dict:
        
        sim_scores = list()
                
        cand_token = nlp(cand_parent)[0]
        
        for net_opt in net_options:
            try:
                sim_scores.append(sim_dict[cand_parent][net_opt])
            except:
                score = cand_token.similarity(nlp(net_opt)[0])
                sim_scores.append(score)
                sim_dict[cand_parent][net_opt] = score

        # Get the index of the most similar word
        sim_idx = np.argmax(sim_scores)
                
        # Recover the float value of the winning word's weight
        sim_weight = float(NET_G.node[net_options[sim_idx]]['weight'])
                
        # Score the similarity times the square root of the word's frequency (weight)
        similarity_score = sim_scores[sim_idx] #* (sim_weight / parent_weight) #np.log(sim_scores[sim_idx]) #+ sim_lp
                        
        return similarity_score
    
    # Otherwise let's score the dependency tags and
    # recursively call this on their children
    else:
        # Prepare to hold scores from multiple branches
        accumulated_scores = []
                
        # Iterate over dependency relations from the parent
        for relation in cand_succ_dict[cand_parent]:
                    
            # Proceed if the NET_graph has this relation:
            try:
                # Get the options from the NET graph branching from this relation type
                child_net_options = NET_succ_dict[relation]
                relation_weight = float(NET_G.node[relation]['weight'])
                print('relation weight:' + str(relation_weight))
            
                # Iterate over the children of each relation
                for cand_child in cand_succ_dict[relation]:
                    score_from_this_node_to_leaves = score_relation_and_children(nlp, cand_G, NET_G, cand_child, child_net_options, cand_succ_dict, NET_succ_dict, sim_dict, relation_weight)
                    if score_from_this_node_to_leaves is not None:
                        accumulated_scores.append(score_from_this_node_to_leaves) # * (np.log(relation_weight) / np.log(parent_weight)))
                    
            except Exception as ex:
                pass
        
        # If we have more than an empty list
        if accumulated_scores != list():
            return sum(accumulated_scores) # logsumexp(accumulated_scores)
            
def compare_candidate_to_NET(nlp, candidate_G, candidate_root, NET_G, net_root, sim_dict):
    
    # Calculate the breadth-first search of the candidate graph
    cand_succ_dict = {par:child_list for (par,child_list)in nx.bfs_successors(candidate_G, candidate_root)}

    # Calculate the breadth-first search of the NET graph
    # (For a speed boost we should do this externallay and pass it in.)
    NET_succ_dict = {par:child_list for (par,child_list)in nx.bfs_successors(NET_G, net_root)}
    
    #print("\n\nNET: {0}".format(net_root))
    
    # Run the results of runnign the recursive score_relation_and_children function on the initial roots
    return score_relation_and_children(nlp, candidate_G, NET_G, candidate_root, [], cand_succ_dict, NET_succ_dict, sim_dict, NET_G.node[net_root]['weight'])

#### Skip: Calculate similarity between one candidate graph and one NET

In [193]:
CANDIDATE_GRAPH_NO = 0
NET_TO_COMPARE = 'PRODUCT'

score = compare_candidate_to_NET(nlp, candidate_roots_and_graphs[CANDIDATE_GRAPH_NO][1], candidate_roots_and_graphs[CANDIDATE_GRAPH_NO][0], log_graphs_dict[NET_TO_COMPARE], NET_TO_COMPARE)

print("\nThe comparison score between G_{2} and the {0} graph is {1}".format(NET_TO_COMPARE, score, CANDIDATE_GRAPH_NO))

NameError: name 'candidate_roots_and_graphs' is not defined

#### Skip: To manually examine the dicts in the comparison above:

In [ ]:
NET_succ_dict = {par:child_list for (par,child_list)in nx.bfs_successors(log_graphs_dict[NET_TO_COMPARE], NET_TO_COMPARE)}

NET_succ_dict

In [ ]:
cand_succ_dict = cand_succ_dict = {par:child_list for (par,child_list)in nx.bfs_successors(candidate_roots_and_graphs[CANDIDATE_GRAPH_NO][1], candidate_roots_and_graphs[CANDIDATE_GRAPH_NO][0])}

cand_succ_dict

### To loop through all the NETs and predict the Max

In [190]:
def predict_max_from_all_nets(cand_root_node, cand_G, NETs_dict, nlp, sim_dict):
    
    # Initialize dict to hold the scores
    similarity_scores_dict = dict()
    
    # Iterate through the NETs
    for net_string, net_graph in NETs_dict.items():
        
        # Score the candidate against each NET
        sim_score = compare_candidate_to_NET(nlp, cand_G, cand_root_node, net_graph, net_string, sim_dict)
        
        # If there is a score for this NET, store it
        if sim_score is not None:
            similarity_scores_dict[net_string] = sim_score
    print(" *** ")
    print(cand_root_node)
    print(similarity_scores_dict)
    
    types_in_likelihood_order = ['ORG', 'GPE', 'PERSON', 'DATE', 'CARDINAL',
                                 'NORP', 'MONEY', 'PERCENT', 'ORDINAL', 'LOC',
                                 'TIME', 'WORK_OF_ART', 'QUANTITY', 'FAC',
                                 'PRODUCT', 'EVENT', 'LAW', 'LANGUAGE']
    
    high_score = float("-inf")
    prediction_so_far = 'NO_MATCH'
    
    for key,value in similarity_scores_dict.items():
        if value > high_score:
            high_score = value
            prediction_so_far = key
        
    return prediction_so_far

def predict_on_doc(doc_row, NETs_dict, nlp, Y_pred, sim_dict):
    
    cluster_dict = dict()
    
    removed_pw = re.sub(r'%pw', 'unk_date', doc_row["document"])
    removed_hypenation = re.sub(r'([a-z])-([a-z])', '\1\2', removed_pw)
    
    filtered_doc = removed_hypenation
    
    ent_dict = graph_candidates_in_doc(doc_row["document_id"],filtered_doc)
    
    for ent_id, (cand_root_node, cand_G, cluster_id) in ent_dict.items():
        try:
            Y_pred.append([ent_id, cluster_dict[cluster_id]])
        except:
            cluster_dict[cluster_id] = predict_max_from_all_nets(cand_root_node, cand_G, NETs_dict, nlp, sim_dict)
            Y_pred.append([ent_id, cluster_dict[cluster_id]])

### Loop through documents to generate Y_Pred entitites

In [5]:
### Import the documents with trace strings removed
documents = pd.read_csv('../data/document.csv')

In [6]:
### Import the pickled NET graphs and calculate and add the log probabilities 
%time

with open('../data/LP_NET_Graphs.pkl', 'rb') as file:
  lp_net_graphs = pickle.load(file)

CPU times: user 3 µs, sys: 1 µs, total: 4 µs
Wall time: 7.87 µs


In [220]:
checkpoint_1 = time.time()

DOC_MIN = 0 
DOC_MAX = 1

# Build the graphs of candidates from the documents.
Y_pred = list()
sim_dict = defaultdict(lambda: dict())

# Loading a spaCy model but disabling parsers to speed up similarity measurements
sim_nlp = spacy.load('en_core_web_lg', disable=['parser', 'tagger', 'ner', 'neuralcoref'])

documents.loc[DOC_MIN:DOC_MAX,:].apply(lambda x: predict_on_doc(x, lp_net_graphs, sim_nlp, Y_pred, sim_dict), axis=1)

print("That took: {0} seconds".format(round(time.time() - checkpoint_1, 4)))


Length of occurence index for doc bc/cctv/00/cctv_0000@0000@cctv@bc@en@on is: 497

Graphing entity: Hong Kong

Graphing entity: Hong Kong

Graphing entity: Hong Kong

Graphing entity: Hong Kong

Graphing entity: Hong Kong

Graphing entity: Hong Kong

Graphing entity: Hong Kong

Graphing entity: Disney

Graphing entity: Disney

Graphing entity: it

Graphing entity: Hong Kong Disneyland

Graphing entity: Hong Kong Disneyland

Graphing entity: Lantau Island

Graphing entity: Lantau Island

Graphing entity: this area

Graphing entity: its

Graphing entity: this area

Graphing entity: Hong Kong International Airport

Graphing entity: its

Graphing entity: Disney

Graphing entity: Disney

Graphing entity: Disney

Graphing entity: Disney

Graphing entity: its

Graphing entity: Disney

Graphing entity: Disney

Graphing entity: the station for Disney

Graphing entity: it

Graphing entity: the Disney subway station

Graphing entity: the park

Graphing entity: the park

Graphing entity: this myst

relation weight:8756.0
relation weight:5038.0
relation weight:11537.0
relation weight:8660.0
relation weight:942.0
relation weight:3738.0
relation weight:1361.0
relation weight:3164.0
relation weight:8761.0
relation weight:6126.0
relation weight:628.0
relation weight:316.0
relation weight:598.0
relation weight:330.0
relation weight:2580.0
relation weight:1054.0
relation weight:109.0
relation weight:79.0
relation weight:771.0
relation weight:409.0
relation weight:1511.0
relation weight:980.0
relation weight:388.0
relation weight:329.0
relation weight:193.0
relation weight:185.0
relation weight:763.0
relation weight:309.0
relation weight:251.0
relation weight:316.0
relation weight:1259.0
relation weight:473.0
the summer of 2005
{'ORDINAL': 0.5752260088920593, 'PERSON': 0.5654570460319519, 'DATE': 0.48497435450553894, 'GPE': 1.0, 'NORP': 0.8012320399284363, 'CARDINAL': 0.4766431152820587, 'ORG': 0.8012320399284363, 'EVENT': 0.5412493944168091, 'WORK_OF_ART': 0.556951105594635, 'MONEY': 0.

relation weight:4637.0
relation weight:11537.0
relation weight:459.0
relation weight:942.0
relation weight:797.0
relation weight:1361.0
relation weight:760.0
relation weight:8761.0
relation weight:548.0
relation weight:628.0
relation weight:90.0
relation weight:598.0
relation weight:85.0
relation weight:2580.0
relation weight:451.0
relation weight:109.0
relation weight:21.0
relation weight:771.0
relation weight:58.0
relation weight:1511.0
relation weight:129.0
relation weight:388.0
relation weight:194.0
relation weight:193.0
relation weight:42.0
relation weight:763.0
relation weight:366.0
relation weight:251.0
relation weight:79.0
relation weight:1259.0
relation weight:118.0
several years
{'ORDINAL': 0.9044975340366364, 'PERSON': 1.1287548542022705, 'DATE': 1.585266888141632, 'GPE': 1.7997552752494812, 'NORP': 1.5388113260269165, 'CARDINAL': 1.4421594142913818, 'ORG': 1.8232534527778625, 'EVENT': 1.5355225205421448, 'WORK_OF_ART': 1.7342053651809692, 'MONEY': 1.5165136456489563, 'LANGU

relation weight:11537.0
relation weight:942.0
relation weight:1361.0
relation weight:8761.0
relation weight:628.0
relation weight:598.0
relation weight:2580.0
relation weight:109.0
relation weight:771.0
relation weight:1511.0
relation weight:388.0
relation weight:193.0
relation weight:763.0
relation weight:251.0
relation weight:1259.0
Disney Corporation
{'ORDINAL': 0.6444936990737915, 'PERSON': 0.6665691137313843, 'DATE': 0.7618895173072815, 'GPE': 1.0, 'NORP': 0.7628620266914368, 'CARDINAL': 0.7628620266914368, 'ORG': 1.0, 'EVENT': 0.7344622015953064, 'WORK_OF_ART': 0.7200257182121277, 'MONEY': 0.6752516627311707, 'LANGUAGE': 0.7344622015953064, 'FAC': 1.0, 'LOC': 1.0, 'QUANTITY': 0.6752516627311707, 'LAW': 0.698711633682251, 'TIME': 1.0, 'PRODUCT': 0.7618895173072815, 'PERCENT': 1.0}
relation weight:20.0
relation weight:139.0
relation weight:1289.0
relation weight:700.0
relation weight:385.0
relation weight:791.0
relation weight:741.0
relation weight:356.0
relation weight:262.0
relat

relation weight:57.0
relation weight:8660.0
relation weight:7247.0
relation weight:3738.0
relation weight:46.0
relation weight:3164.0
relation weight:189.0
relation weight:6126.0
relation weight:4.0
relation weight:316.0
relation weight:3.0
relation weight:330.0
relation weight:5.0
relation weight:1054.0
relation weight:88.0
relation weight:79.0
relation weight:1.0
relation weight:409.0
relation weight:29.0
relation weight:980.0
relation weight:12.0
relation weight:329.0
relation weight:2.0
relation weight:185.0
relation weight:43.0
relation weight:309.0
relation weight:6.0
relation weight:316.0
relation weight:4.0
relation weight:473.0
first
{'ORDINAL': 1.0, 'PERSON': 0.4959487318992615, 'DATE': 0.5851811170578003, 'GPE': 0.48871877789497375, 'NORP': 1.0, 'CARDINAL': 0.5851811170578003, 'ORG': 0.46426916122436523, 'EVENT': 0.7658557295799255, 'WORK_OF_ART': 0.47963425517082214, 'MONEY': 0.4632461667060852, 'LANGUAGE': 0.4213261902332306, 'FAC': 0.5413610935211182, 'LOC': 0.46659880876

...

relation weight:1363.0
relation weight:2765.0
relation weight:767.0
relation weight:5038.0
relation weight:1237.0
relation weight:8660.0
relation weight:230.0
relation weight:3738.0
relation weight:166.0
relation weight:3164.0
relation weight:559.0
relation weight:6126.0
relation weight:12.0
relation weight:316.0
relation weight:124.0
relation weight:330.0
relation weight:272.0
relation weight:1054.0
relation weight:36.0
relation weight:409.0
relation weight:18.0
relation weight:980.0
relation weight:31.0
relation weight:329.0
relation weight:2.0
relation weight:185.0
relation weight:48.0
relation weight:309.0
relation weight:75.0
relation weight:316.0
relation weight:722.0
relation weight:473.0
Yangquan , Shanxi
{'ORDINAL': 0.4281756281852722, 'PERSON': 0.6090659499168396, 'DATE': 0.49977290630340576, 'GPE': 0.8760979175567627, 'NORP': 0.6553283333778381, 'CARDINAL': 0.6582672595977783, 'ORG': 0.4817699193954468, 'EVENT': 0.5099815726280212, 'WORK_OF_ART': 0.4724672734737396, 'MONEY':

relation weight:12265.0
relation weight:218.0
relation weight:2606.0
relation weight:987.0
relation weight:626.0
relation weight:6841.0
relation weight:142.0
relation weight:144.0
relation weight:74.0
relation weight:18.0
relation weight:109.0
relation weight:101.0
relation weight:28.0
relation weight:50.0
relation weight:29.0
relation weight:129.0
relation weight:290.0
More than 60 years
{'ORDINAL': 0.5438535809516907, 'PERSON': 1.0, 'DATE': 0.3213326930999756, 'GPE': 0.5390946865081787, 'NORP': 1.0, 'CARDINAL': 0.6320889592170715, 'ORG': 0.5253705382347107, 'EVENT': 1.0, 'WORK_OF_ART': 1.0, 'MONEY': 0.5960456728935242, 'LANGUAGE': 0.370281845331192, 'FAC': 0.6320889592170715, 'LOC': 0.4901807904243469, 'QUANTITY': 1.0, 'LAW': 0.5438535809516907, 'TIME': 0.39059463143348694, 'PRODUCT': 0.5866234302520752, 'PERCENT': 0.5960456728935242}
relation weight:1763.0
relation weight:7.0
relation weight:23.0
relation weight:576.0
relation weight:1004.0
relation weight:253.0
relation weight:57.0

relation weight:46.0
relation weight:3164.0
relation weight:189.0
relation weight:6126.0
relation weight:4.0
relation weight:316.0
relation weight:3.0
relation weight:330.0
relation weight:5.0
relation weight:1054.0
relation weight:88.0
relation weight:79.0
relation weight:1.0
relation weight:409.0
relation weight:29.0
relation weight:980.0
relation weight:12.0
relation weight:329.0
relation weight:2.0
relation weight:185.0
relation weight:43.0
relation weight:309.0
relation weight:6.0
relation weight:316.0
relation weight:4.0
relation weight:473.0
Japanese
{'ORDINAL': 0.31593090295791626, 'PERSON': 0.3408399522304535, 'DATE': 0.4941527545452118, 'GPE': 0.5103696584701538, 'NORP': 0.40713998675346375, 'CARDINAL': 0.4895731508731842, 'ORG': 0.49922439455986023, 'EVENT': 0.4405209720134735, 'WORK_OF_ART': 0.31673523783683777, 'MONEY': 0.41451495885849, 'LANGUAGE': 0.2573383152484894, 'FAC': 0.4826021194458008, 'LOC': 0.47879207134246826, 'QUANTITY': 0.47879207134246826, 'LAW': 0.27550426

relation weight:24.0
relation weight:1840.0
relation weight:15.0
relation weight:527.0
relation weight:117.0
relation weight:72.0
relation weight:723.0
relation weight:33.0
relation weight:231.0
relation weight:11.0
relation weight:97.0
relation weight:8.0
relation weight:3.0
relation weight:412.0
relation weight:396.0
relation weight:2.0
relation weight:17.0
relation weight:1.0
relation weight:2.0
relation weight:71.0
relation weight:3.0
relation weight:3.0
relation weight:4.0
relation weight:98.0
relation weight:61.0
relation weight:54.0
relation weight:111.0
relation weight:77.0
relation weight:1.0
relation weight:7.0
relation weight:288.0
relation weight:37.0
60 years ago
{'ORDINAL': 0.809291422367096, 'PERSON': 1.090437412261963, 'DATE': 1.497912049293518, 'GPE': 0.47068777680397034, 'NORP': 0.9946688711643219, 'CARDINAL': 1.086488276720047, 'ORG': 0.6237398982048035, 'WORK_OF_ART': 0.9469535052776337, 'MONEY': 1.5456973910331726, 'LANGUAGE': 0.9452270865440369, 'FAC': 0.610371820

relation weight:8803.0
relation weight:132.0
relation weight:35.0
relation weight:336.0
relation weight:24.0
relation weight:13.0
relation weight:13.0
relation weight:20.0
relation weight:760.0
relation weight:12.0
relation weight:1.0
relation weight:37.0
relation weight:8.0
relation weight:3.0
relation weight:19.0
relation weight:286.0
relation weight:7.0
relation weight:90.0
relation weight:31.0
relation weight:55.0
relation weight:6.0
relation weight:91.0
relation weight:42.0
relation weight:70.0
March 19
{'ORDINAL': 0.26527637243270874, 'PERSON': 0.16976192593574524, 'DATE': 0.29529666900634766, 'NORP': 0.4389230012893677, 'CARDINAL': 0.5028860569000244, 'ORG': 0.24715417623519897, 'EVENT': 0.4135019779205322, 'WORK_OF_ART': 0.4777846932411194, 'MONEY': 0.3284265697002411, 'FAC': 0.24501238763332367, 'LOC': 0.11813230067491531, 'QUANTITY': 0.18880674242973328, 'LAW': 1.0, 'TIME': 0.291050523519516, 'PRODUCT': 0.340106338262558}
relation weight:2.0
relation weight:40.0
relation weig

#### Confirming the number of entities match for the first two documents

In [195]:
len(Y_pred)

714

In [196]:
doc0 = nlp(removed_hypenation)
doc1 = nlp(documents.loc[1,"document"])

len(doc0.ents) + len(doc1.ents)

716

### Comparing Y_Pred and Y_True on the first two documents

In [221]:
ypred_df = pd.DataFrame(Y_pred, columns = ['entity_id', 'prediction']).set_index('entity_id')

In [222]:
#Save ypred to file
ypred_df.to_csv('../data/Y_pred.csv')

In [15]:
with open('../data/Y_true.pkl', 'rb') as file:
  ytrue_df = pickle.load(file)

In [223]:
ytrue_df_filtered = ytrue_df.filter(like="bc/cctv/00/cctv_0000@0000@cctv@bc@en@on",axis=0).sort_values(by=['sentence_index', 'start_word_index'])

ymerged_df = ypred_df.merge(ytrue_df_filtered,how='inner',on='entity_id')
ymerged_df.shape

(248, 7)

In [224]:
ymerged_df.head()

,prediction,document_id,type,sentence_index,start_word_index,end_word_index,string
entity_id,,,,,,,
bc/cctv/00/cctv_0000@0000@cctv@bc@en@on:1:4,GPE,bc/cctv/00/cctv_0000@0000@cctv@bc@en@on,DATE,0,1,4,the summer of 2005
bc/cctv/00/cctv_0000@0000@cctv@bc@en@on:23:24,LOC,bc/cctv/00/cctv_0000@0000@cctv@bc@en@on,GPE,0,23,24,Hong Kong
bc/cctv/00/cctv_0000@0000@cctv@bc@en@on:41:42,LOC,bc/cctv/00/cctv_0000@0000@cctv@bc@en@on,GPE,1,14,15,Hong Kong
bc/cctv/00/cctv_0000@0000@cctv@bc@en@on:54:54,ORDINAL,bc/cctv/00/cctv_0000@0000@cctv@bc@en@on,ORDINAL,2,3,3,fifth
bc/cctv/00/cctv_0000@0000@cctv@bc@en@on:55:55,LOC,bc/cctv/00/cctv_0000@0000@cctv@bc@en@on,ORG,2,4,4,Disney


In [225]:
matches = ymerged_df.loc[:,["prediction","type"]].apply(lambda x: x["prediction"] == x["type"], axis=1)
accuracy = sum(matches) / ymerged_df.shape[0]
print("Accuracy: {0}".format(accuracy))

Accuracy: 0.4032258064516129


In [186]:
ytrue_df.groupby("type").size().sort_values()

type
LANGUAGE         415
LAW              568
EVENT           1273
PRODUCT         1297
FAC             1440
QUANTITY        1583
WORK_OF_ART     1652
TIME            2290
LOC             2692
ORDINAL         2738
PERCENT         4866
MONEY           6429
NORP           11610
CARDINAL       13638
DATE           23807
PERSON         27351
GPE            28142
ORG            29963
dtype: int64